<a href="https://colab.research.google.com/github/xerathul/python/blob/master/tf31cnn_mnist_subclassing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# CNN : 이미지의 특징을 뽑아 크기를 줄인 후 마지막에 1차원 배열로 만든 후 Dense에게 전달하는 방식
# MNIST dataset을 사용
# subclassing model api 방법
import tensorflow as tf
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras import datasets, Model

(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

train_images = x_train.reshape((60000, 28, 28, 1))
train_images =  train_images / 255.0
test_images = x_test.reshape((10000, 28, 28, 1))
test_images =  test_images / 255.0
train_labels = y_train
test_labels = y_test

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [11]:
# tf.data.Dataset.from_tensor_slices() : 입력 파이프라인을 가능하게 함. 데이터를 골고루 섞기 등의 작업이 가능
# Dataset.from_tensor_slices() 경험해 보기 ----------
import numpy as np
x = np.random.sample((5, 2))
print(x)
dset = tf.data.Dataset.from_tensor_slices(x)
print(dset)
dset = tf.data.Dataset.from_tensor_slices(x).shuffle(10000).batch(3)
print(dset)
for a in dset:
    print(a)
# ---------------------------------------------------


[[0.08050424 0.44565513]
 [0.41209626 0.74065825]
 [0.92934785 0.95255907]
 [0.26258291 0.58926741]
 [0.78936217 0.09917395]]
<TensorSliceDataset element_spec=TensorSpec(shape=(2,), dtype=tf.float64, name=None)>
<BatchDataset element_spec=TensorSpec(shape=(None, 2), dtype=tf.float64, name=None)>
tf.Tensor(
[[0.92934785 0.95255907]
 [0.41209626 0.74065825]
 [0.08050424 0.44565513]], shape=(3, 2), dtype=float64)
tf.Tensor(
[[0.78936217 0.09917395]
 [0.26258291 0.58926741]], shape=(2, 2), dtype=float64)


In [12]:
# MNIST의 학습 데이터(60000, 28, 28)가 입력되면 6만 개의 slice를 만들고 각 slice는 28 * 28 형태로 만들기
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(60000).batch(28)
print(train_ds)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(28)
print(test_ds)

<BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


In [13]:
# subclassing model
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(filters=32, kernel_size=[3,3], padding='valid', activation='relu')
        self.pool1 = MaxPool2D((2,2))
        self.conv2 = Conv2D(filters=32, kernel_size=[3,3], padding='valid', activation='relu')
        self.pool2 = MaxPool2D((2,2))
        self.flatten = Flatten(dtype='float32')
        self.d1 = Dense(32, activation='relu')
        self.drop1 = Dropout(rate=0.2)
        self.d2 = Dense(10, activation='softmax')

    def call(self, inputs):
        net = self.conv1(inputs)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.flatten(net)
        net = self.d1(net)
        net = self.drop1(net)
        net = self.d2(net)
        return net

model = MyModel()
temp_inputs = tf.keras.Input(shape = (28, 28, 1))
model(temp_inputs)



<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'my_model_1')>

In [14]:
from keras import callbacks
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=128, epochs=5, verbose=2, 
          use_multiprocessing=True, workers=2)
score = model.evaluate(test_images, test_labels)
print('loss : ', score[0])
print('acc : ', score[1])

Epoch 1/5
469/469 - 13s - loss: 0.4579 - accuracy: 0.8579 - 13s/epoch - 28ms/step
Epoch 2/5
469/469 - 2s - loss: 0.1642 - accuracy: 0.9492 - 2s/epoch - 3ms/step
Epoch 3/5
469/469 - 2s - loss: 0.1242 - accuracy: 0.9618 - 2s/epoch - 3ms/step
Epoch 4/5
469/469 - 2s - loss: 0.1043 - accuracy: 0.9682 - 2s/epoch - 3ms/step
Epoch 5/5
469/469 - 2s - loss: 0.0900 - accuracy: 0.9722 - 2s/epoch - 3ms/step
313/313 [==============================] - 1s 3ms/step - loss: 0.0395 - accuracy: 0.9863
loss :  0.03949269279837608
acc :  0.986299991607666
